# Q&A about the document with source

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain/blob/main/examples/QA_for_document_with_source.ipynb)

In [ ]:
%%capture
!pip install langchain
!pip install openai
!pip install faiss-cpu
!pip install tiktoken

In [5]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain.llms import OpenAI

In [8]:
file_path = "/home/jovyan/langchain/data/sample.txt"


#  データの用意
with open(file_path) as f:
    dragonball_txt = f.read()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_text(dragonball_txt)


query = "Kaggle社内Meet upとはどんなものですか?"

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(
    texts, 
    embeddings,
    metadatas=[{"source": i} for i in range(len(texts))])
docs = docsearch.similarity_search(query)

In [9]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

# ソース付きの質問応答の実行
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Kaggle社内Meet upとは、Kaggleに参加したことがある社員や参加したことがないけれど興味がある社員、Kaggleに限らず技術コンペについて情報収集したい社員が、交流できるように企画されたイベントです。オフラインとオンラインの同時開催で、Kaggleに参加する動機や参加するコンペの選択方法、戦績、Kaggleの取り組み方などを発表し、懇親会を行うイベントです。\nSOURCES: 0, 1, 2, 3'}

In [10]:
# 参照したソースの表示
print(texts[0])
print(texts[1])
print(texts[2])
print(texts[3])

Kaggle社内Meet upとは？
「Kaggle」については、ご存知の方も多いと思いますが、企業や組織・団体から投稿された課題とデータをもとに、解決に適した機械学習のモデルを構築して精度を競い合うコンペ式のサービスを言います。
当社にはデータサイエンティストが約200名在籍していることもあり、データ分析の技術を研鑽するため、プライベートでKaggleに親しむKagglerが数多く在籍しています。

今回のMeet upは、「Kaggleに参加したことがある」「参加したことはないけれど興味がある」「Kaggleに限らず技術コンペについて情報収集したい」という社員同志が、交流できるように企画されたイベントです！

オフラインとオンラインの同時開催
感染症対策の考慮もしつつ、当日は本社オフィスの広い会場（セミナールーム）にて、オフラインとオンライン、どちらでも参加可能なハイブリッド形式で開催されました。
悪天候にも関わらずセミナールームには約30名が集合し、オンラインでは社員の他に新卒の内定者も加わって、総勢90名が参加する大きなイベントとなりました。
中には、懇親会を楽しみに、お客様先などから本社オフィスへ移動してくる参加者もいました。

当日のコンテンツ
前半は、Kaggle歴1年目からKaggle master（！）まで幅広い経歴の4名によるライトニングトークが行われました。

Kaggleに参加する動機、参加するコンペの選択方法、戦績、Kaggleの取り組み方、参加して得られたこと、苦しみ…など、幅広い話題が思い思いの形で発表され、笑いが起きる場面もしばしば。発表後には様々な質疑が飛び交いました。

後半は、ピザなどの軽食とドリンクなどを囲み、懇親会が開催されました。皆さん、和気あいあいと打ち解けていました。


参加者の感想
普段あまりKaggleの活動について話す機会がないため、とても充実した時間を過ごすことができた。
楽しそうに取り組んでいる様子が伝わってきて自分のモチベーションにもつながった。
まだKaggleに取り組んだことがなく、コンペの選び方などがさっぱり分からなかったので、ライトニングトークが非常に参考になった。
お客様への報告資料のようにまとめられていたところが、ブレインパッドっぽくて面白かったです。

などなど…。これらの感想のとおり、「と